## Install Dependencies

In [1]:
!pip install -q transformers accelerate bitsandbytes datasets sentence-transformers pandas tqdm
!pip install -q langchain langchain-community langchain-huggingface langchain-chroma chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 13.9 MB/s eta 0:

In [2]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from datasets import load_dataset


In [3]:
RETRIEVAL_K = 3
EMBEDDING_MODEL_NAME = "intfloat/e5-base-v2"
SAMPLES_FOR_DB = 1000
SAMPLES_FOR_TEST = 100
MODEL_ID = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

print(f"Using model: {MODEL_ID}")



Using model: HuggingFaceTB/SmolLM2-1.7B-Instruct


## Evaluation Technique

In [4]:

class Evaluator:

    def normalize_text(text):
        text = str(text).lower().strip()
        return ' '.join(text.split())

    def is_correct(prediction, ground_truth):
        pred = Evaluator.normalize_text(prediction)
        truth = Evaluator.normalize_text(ground_truth)
        return truth in pred


## Preparing the dataset

In [5]:
#
def prepare_data(dataset, start_idx, count):
    data = []
    subset = dataset.select(range(start_idx, start_idx + count))

    for row in subset:
        answer = row['answer']['aliases'][0]
        snippets = row['search_results']['search_context']
        context = " ||| ".join(snippets) if snippets else ""

        data.append({
            "question": row['question'],
            "answer": answer,
            "context": context
        })

    return pd.DataFrame(data)


## Loading model

In [6]:

def load_model_and_tokenizer():
    print(f"Loading model...")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
    )

    return model, tokenizer


In [7]:

def generate_answer(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.5,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if prompt in full_text:
        answer = full_text[len(prompt):].strip()
    else:
        answer = full_text.strip()

    return answer[:300].strip()



## Building Vector Database

In [8]:

def build_vector_database(train_df):

    documents = []
    for _, row in train_df.iterrows():
        snippets = str(row['context']).split(" ||| ")
        for snippet in snippets:
            if len(snippet) > 20:
                documents.append(Document(page_content=snippet[:800]))

    documents = documents[:3000]

    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        encode_kwargs={'normalize_embeddings': True}
    )

    vectordb = Chroma.from_documents(
        documents,
        embeddings,
        collection_name="trivia_qa"
    )

    return vectordb


In [9]:

def create_prompt(question, context):
    return f"{context[:300]}\n\nQ: {question}\nA:"


def run_evaluation(model, tokenizer, test_df, retriever):
    evaluator = Evaluator()
    results = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
        question = row['question']
        ground_truth = row['answer']

        query = f"query: {question}"
        retrieved_docs = retriever.invoke(query)
        context = "\n".join([doc.page_content[:250] for doc in retrieved_docs])

        prompt = create_prompt(question, context)
        prediction = generate_answer(model, tokenizer, prompt)

        correct = evaluator.is_correct(prediction, ground_truth)

        results.append({
            "Question": question,
            "Ground_Truth": ground_truth,
            "Prediction": prediction,
            "Correct": correct
        })

    return pd.DataFrame(results)



## Evaluation

In [10]:


def main():
    print("RAG TriviaQA Evaluation\n")

    # Load dataset
    print("Loading dataset...")
    dataset = load_dataset("mandarjoshi/trivia_qa", "rc", split="train")

    train_df = prepare_data(dataset, 0, SAMPLES_FOR_DB)
    test_df = prepare_data(dataset, 0, SAMPLES_FOR_TEST)
    print(f"Prepared {len(train_df)} training and {len(test_df)} test samples\n")

    # Load model
    model, tokenizer = load_model_and_tokenizer()

    # Build retrieval system
    vectordb = build_vector_database(train_df)
    retriever = vectordb.as_retriever(search_kwargs={"k": RETRIEVAL_K})

    # Run evaluation
    print("\nStarting evaluation...")
    results_df = run_evaluation(model, tokenizer, test_df, retriever)

    # Calculate and display accuracy
    accuracy = results_df['Correct'].mean() * 100

    # Save results
    results_df.to_csv("rag_triviaqa_results.csv", index=False)

    print(f"\nFinal Accuracy: {accuracy:.2f}%")


if __name__ == "__main__":
    main()

RAG TriviaQA Evaluation

Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]


Starting evaluation...


Evaluating: 100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


Final Accuracy: 14.00%
